In [1]:
#default_exp modules.loss

# The loss function
> We will define our loss function in this file. For general guidance we used the original pointpillars paper.
>

In [2]:
import sys
sys.path.append("/home/qhs67/git/bachelorthesis_sven_thaele/code/")

In [3]:
#export
import logging
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

logger = logging.getLogger(__name__)

In [4]:
#export
class PointPillarsLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        logger.info("Initializing PointPillarsLoss module...")
        super(PointPillarsLoss, self).__init__()

    def forward(self, preds: list, writer: SummaryWriter = None, epoch: int = 0, i: int = 0) -> float:
        """
        Calculates the resulting, overall loss function. It consists of the occupancy loss for positive as well as
        negative matches, and the regressional loss, which is intern calculated by positive match attributes.

        :param preds: A list containing the matched tensors. The list has the following structure:
                      [pred_occ(nb_matched_boxes),
                       pred_cls(nb_matched_boxes),
                       pred_head(nb_matched_boxes),
                       pred_box(nb_matched_boxes, nb_attributes=7),
                       gt_boxes(nb_matched_boxes, nb_attributes=7),
                       neg_matches(nb_negative_match)]

                       gt_boxes are the gt_boxes matched to each pred_box with iou >= pos_iou_threshold
                       neg_match gives the pred_occ for every box with a iou match lower than the neg_iou_threshold

        :return: loss value as integer
        """
        pred_occ, pred_cls, pred_head, pred_boxes, gt_boxes, neg_matches = preds

        L_occ_pos = self.pos_occ_loss(preds)
        L_occ_neg = self.neg_occ_loss(preds)
        L_reg = self.reg_loss(preds, writer, epoch, i)
        N_pos = pred_boxes.shape[0]
        N_neg = neg_matches.shape[0]

        # defined in VoxelNet paper
        alpha = 1.5
        beta = 1

        if N_pos == 0 and N_neg == 0:
            raise ValueError("Neither positive nor negative Matches.")
            #loss = 0.0

        elif N_pos == 0:
            loss = beta * 1/N_neg * L_occ_neg

            if writer is not None:
                writer.add_scalar("Epoch {}/L_occ_neg".format(epoch), L_occ_neg/N_neg, i)
                writer.add_scalar("Epoch {}/L_occ_pos".format(epoch), 0, i)
                writer.add_scalar("Epoch {}/L_reg".format(epoch), 0, i)

        elif N_neg == 0:
            loss = alpha * 1/N_pos * L_occ_pos + L_reg

            if writer is not None:
                writer.add_scalar("Epoch {}/L_occ_neg".format(epoch), 0, i)
                writer.add_scalar("Epoch {}/L_occ_pos".format(epoch), L_occ_pos/N_pos, i)
                writer.add_scalar("Epoch {}/L_reg".format(epoch), L_reg, i)

        else:
            loss = alpha * 1/N_pos * L_occ_pos + beta * 1/N_neg * L_occ_neg + L_reg

            if writer is not None:
                writer.add_scalar("Epoch {}/L_occ_neg".format(epoch), L_occ_neg/N_neg, i)
                writer.add_scalar("Epoch {}/L_occ_pos".format(epoch), L_occ_pos/N_pos, i)
                writer.add_scalar("Epoch {}/L_reg".format(epoch), L_reg, i)

        return loss


    def reg_loss(self, preds: list, writer: SummaryWriter = None, epoch: int = 0, i: int = 0) -> float:
        """
        Calculates the regressional loss. The function is defined in the PointPillars paper. This here includes
        localisation loss, class loss and directional loss.

        :param preds: A list containing the matched tensors. The list has the following structure:
                      [pred_occ(nb_matched_boxes),
                       pred_cls(nb_matched_boxes),
                       pred_head(nb_matched_boxes),
                       pred_box(nb_matched_boxes, nb_attributes=7),
                       gt_boxes(nb_matched_boxes, nb_attributes=7),
                       neg_matches(nb_negative_match)]

                       gt_boxes are the gt_boxes matched to each pred_box with iou >= pos_iou_threshold
                       neg_match gives the pred_occ for every box with a iou match lower than the neg_iou_threshold

        :return: loss value as integer
        """
        logger.debug("Calculating reg_loss...")

        _, _, _, pred_boxes, _, _ = preds
        N_pos = pred_boxes.shape[0]

        # no positive matches
        if N_pos == 0:
            return 0.0

        # definded by the PointPillars paper
        beta_loc = 2
        beta_cls = 1
        beta_dir = 0.2

        L_loc = self.loc_loss(preds)
        L_cls = self.cls_loss(preds)
        L_dir = self.dir_loss(preds)

        L_reg = 1/N_pos * (beta_loc * L_loc + beta_cls * L_cls + beta_dir * L_dir)

        if writer is not None:
            writer.add_scalar("Epoch {}/L_loc".format(epoch), L_loc/N_pos, i)
            writer.add_scalar("Epoch {}/L_cls".format(epoch), L_cls/N_pos, i)
            writer.add_scalar("Epoch {}/L_dir".format(epoch), L_dir/N_pos, i)

        return L_reg

    def loc_loss(self, preds: list) -> float:
        """
        The localisation loss is calculated. A set of individual equations is used, which later on are added together.
        The exact equations can be obtained from the PointPillars paper.

        :param preds: A list containing the matched tensors. The list has the following structure:
                      [pred_occ(nb_matched_boxes),
                       pred_cls(nb_matched_boxes),
                       pred_head(nb_matched_boxes),
                       pred_boxes(nb_matched_boxes, nb_attributes=7),
                       gt_boxes(nb_matched_boxes, nb_attributes=7),
                       neg_matches(nb_negative_match)]

                       gt_boxes are the gt_boxes matched to each pred_box with iou >= pos_iou_threshold
                       neg_match gives the pred_occ for every box with a iou match lower than the neg_iou_threshold

        :return: loss value as integer
        """
        _, _, _, pred_boxes, gt_boxes, _ = preds
        xa, ya, za, ha, wa, la, thetaa = torch.split(pred_boxes, 1, dim=1)
        xg, yg, zg, hg, wg, lg, thetag = torch.split(gt_boxes, 1, dim=1)

        diagonal = torch.sqrt(la**2 + wa**2)
        dx = (xg - xa) / diagonal
        dy = (yg - ya) / diagonal
        dz = (zg - za) / ha
        dl = torch.log(lg / la)
        dw = torch.log(wg / wa)
        dh = torch.log(hg / ha)
        dtheta = torch.sin(thetaa - thetag)

        calc_values = [dx, dy, dz, dl, dw, dh, dtheta]
        smooth_losses = []

        # SmoothL1Loss() every individual loss in location loss
        for value in calc_values:

            loss_func = torch.nn.SmoothL1Loss(reduction='sum')
            smooth_losses.append(loss_func(value, torch.zeros_like(value)))

        return sum(smooth_losses)

    def cls_loss(self, preds: list) -> float:
        """
        Class loss is calculated. We use focal loss.

        :param preds: A list containing the matched tensors. The list has the following structure:
                      [pred_occ(nb_matched_boxes),
                       pred_cls(nb_matched_boxes),
                       pred_head(nb_matched_boxes),
                       pred_boxes(nb_matched_boxes, nb_attributes=7),
                       gt_boxes(nb_matched_boxes, nb_attributes=7),
                       neg_matches(nb_negative_match)]

                       gt_boxes are the gt_boxes matched to each pred_box with iou >= pos_iou_threshold
                       neg_match gives the pred_occ for every box with a iou match lower than the neg_iou_threshold

        :return: loss value as integer
        """
        _, pred_cls, _, _, _, _ = preds

        target = torch.zeros_like(pred_cls, device="cuda:0")
        focal_loss = WeightedFocalLoss()

        return focal_loss(pred_cls, target)

    def dir_loss(self, preds: list) -> float:
        """
        Directional loss is calculated.

        :param preds: A list containing the matched tensors. The list has the following structure:
                      [pred_occ(nb_matched_boxes),
                       pred_cls(nb_matched_boxes),
                       pred_head(nb_matched_boxes),
                       pred_box(nb_matched_boxes, nb_attributes=7),
                       gt_boxes(nb_matched_boxes, nb_attributes=7),
                       neg_matches(nb_negative_match)]

                       gt_boxes are the gt_boxes matched to each pred_box with iou >= pos_iou_threshold
                       neg_match gives the pred_occ for every box with a iou match lower than the neg_iou_threshold

        :return: loss value as float
        """
        # kitti convention: ry == 0 if object is aligned with x-axis and pointing right
        _, _, pred_head, _, gt_boxes, _ = preds

        # if the yaw rotation around the z-axis of the ground truth is higher than zero,
        # the result is positive; otherwise, it is negative
        # theta_gt      target
        #   < 0           0
        #   >= 0           +1

        gt_theta = gt_boxes[:,6]
        theta_greater_zero = gt_theta >= 0
        target_greater_zero = theta_greater_zero.float()
        target = torch.zeros_like(pred_head, device='cuda:0') + target_greater_zero

        ce_loss = nn.BCEWithLogitsLoss(reduction='sum')
        return ce_loss(pred_head, target)

    def pos_occ_loss(self, preds: list) -> float:
        """
        Calculates the binary entropy loss for positive matches. Therefore pred_occ tensor is used.

        :param preds: A list containing the matched tensors. The list has the following structure:
                      [pred_occ(nb_matched_boxes),
                       pred_cls(nb_matched_boxes),
                       pred_head(nb_matched_boxes),
                       pred_box(nb_matched_boxes, nb_attributes=7),
                       gt_boxes(nb_matched_boxes, nb_attributes=7),
                       neg_matches(nb_negative_match)]

                       gt_boxes are the gt_boxes matched to each pred_box with iou >= pos_iou_threshold
                       neg_match gives the pred_occ for every box with a iou match lower than the neg_iou_threshold

        :return: loss value as integer
        """
        pred_occ, _, _, _, _, _ = preds

        if pred_occ.shape[0] == 0:
            return 0.0

        target = torch.ones_like(pred_occ, device='cuda:0')

        #(binary) cross entropy loss
        bce_loss = nn.BCEWithLogitsLoss(reduction='sum')
        return bce_loss(pred_occ, target)

    def neg_occ_loss(self, preds: list) -> float:
        """
        Calculates the binary cross entropy loss for negative matches. Therefore neg_matches tensor is used.

        :param preds: A list containing the matched tensors. The list has the following structure:
                      [pred_occ(nb_matched_boxes),
                       pred_cls(nb_matched_boxes),
                       pred_head(nb_matched_boxes),
                       pred_box(nb_matched_boxes, nb_attributes=7),
                       gt_boxes(nb_matched_boxes, nb_attributes=7),
                       neg_matches(nb_negative_match)]

                       gt_boxes are the gt_boxes matched to each pred_box with iou >= pos_iou_threshold
                       neg_match gives the pred_occ for every box with a iou match lower than the neg_iou_threshold

        :return: loss value as integer
        """
        _, _, _, _, _, neg_matches = preds

        if neg_matches.shape[0] == 0:
            return 0.0

        target = torch.zeros_like(neg_matches, device='cuda:0')

        #(binary) cross entropy loss
        bce_loss = nn.BCEWithLogitsLoss(reduction='sum')

        return bce_loss(neg_matches, target)


In [5]:
#export
class WeightedFocalLoss(nn.Module):
    "Non weighted version of Focal Loss"
    def __init__(self, alpha=0.25, gamma=2):
        super(WeightedFocalLoss, self).__init__()
        self.alpha = torch.tensor([alpha, 1-alpha], device=torch.device("cuda"))
        self.gamma = gamma

    def forward(self, inputs: torch.Tensor, targets: torch.Tensor) -> float:
        """Returns the focal loss for inputs"""
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        targets = targets.type(torch.long)
        at = self.alpha.gather(0, targets.data.view(-1))
        pt = torch.exp(-BCE_loss)
        F_loss = at*(1-pt)**self.gamma * BCE_loss
        return F_loss.sum()

In [6]:
a = torch.rand(100).cuda() * 2 - 1
target = torch.zeros_like(a).cuda()

focal_loss = WeightedFocalLoss()
focal_loss(a, target)

tensor(5.6516, device='cuda:0')

In [24]:
input = [
    torch.rand((1000)).cuda(),
    torch.rand((1000)).cuda(),
    torch.rand((1000)).cuda(),
    torch.rand((1000, 7)).cuda(),
    torch.rand((1000, 7)).cuda(),
    torch.rand((1000)).cuda(),
    (torch.rand((1000)).cuda() >=0.5).type(torch.IntTensor),
]
ppLoss = PointPillarsLoss()

out = ppLoss._exlude_masked(input)
for tens in out:
    print(tens.shape)

torch.Size([462])
torch.Size([462])
torch.Size([462])
torch.Size([462, 7])
torch.Size([462, 7])
torch.Size([462])


In [7]:
a = torch.rand(10, 7)
print(a, a.shape)

a1, a2, a3, a4, a5, a6, a7 = torch.split(a, 1, dim=1)
print(a1, a2, a3, a4)


tensor([[0.4986, 0.0163, 0.1210, 0.7441, 0.5782, 0.6981, 0.7556],
        [0.4181, 0.4219, 0.4841, 0.3865, 0.1049, 0.4564, 0.0951],
        [0.3956, 0.9177, 0.3972, 0.6727, 0.4713, 0.6087, 0.1112],
        [0.4597, 0.6505, 0.8173, 0.2113, 0.7539, 0.2706, 0.9028],
        [0.0316, 0.6660, 0.7183, 0.6688, 0.3348, 0.2928, 0.3328],
        [0.9797, 0.5109, 0.4738, 0.3201, 0.3925, 0.3667, 0.9568],
        [0.9949, 0.8854, 0.2696, 0.3935, 0.9701, 0.8669, 0.7429],
        [0.8059, 0.3103, 0.3384, 0.9785, 0.5852, 0.9211, 0.6828],
        [0.8642, 0.1476, 0.8894, 0.5632, 0.3926, 0.8802, 0.2078],
        [0.3242, 0.7258, 1.0000, 0.6910, 0.5313, 0.8182, 0.7950]]) torch.Size([10, 7])
tensor([[0.4986],
        [0.4181],
        [0.3956],
        [0.4597],
        [0.0316],
        [0.9797],
        [0.9949],
        [0.8059],
        [0.8642],
        [0.3242]]) tensor([[0.0163],
        [0.4219],
        [0.9177],
        [0.6505],
        [0.6660],
        [0.5109],
        [0.8854],
        [0.3

In [8]:
input = [
    torch.rand((1000)).cuda(),
    torch.rand((1000)).cuda(),
    torch.rand((1000)).cuda(),
    torch.rand((1000, 7)).cuda(),
    torch.rand((1000, 7)).cuda(),
    torch.rand((500)).cuda()
]

ppLoss = PointPillarsLoss()
loss = ppLoss(input)

loss

TypeError: forward() missing 3 required positional arguments: 'writer', 'epoch', and 'i'